In [ ]:
import pandas as pd
# import pandas_ta as ta

# Load data
df = pd.read_csv("AAPL.csv", sep=",")

# VWAP requires the DataFrame index to be a DatetimeIndex.
# Replace "datetime" with the appropriate column from your DataFrame
df.set_index(pd.DatetimeIndex(df["Date"]), inplace=True)

# Calculate Returns and append to the df DataFrame
df.ta.log_return(cumulative=True, append=True)
df.ta.percent_return(cumulative=True, append=True)

# New Columns with results
df.columns

# Take a peek
df.tail()

# vv Continue Post Processing vv

In [1]:
import pandas as pd

In [9]:
import datetime

In [2]:
df = pd.DataFrame({'date': ['2018-08-09 11:10:55','2019-03-02 13:15:21']})

In [3]:
df

,date
0,2018-08-09 11:10:55
1,2019-03-02 13:15:21


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    2 non-null      object
dtypes: object(1)
memory usage: 144.0+ bytes


In [5]:
df['date'] = pd.to_datetime(df['date'])

In [6]:
df

,date
0,2018-08-09 11:10:55
1,2019-03-02 13:15:21


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 144.0 bytes


In [10]:
datetime.datetime.now()

datetime.datetime(2021, 7, 3, 8, 18, 25, 693631)

In [14]:
# First importing the needed libraries:

import pandas as pd
import numpy as np
import datetime


import os
import requests
import json
import base64
import time
import base64
import hmac
import hashlib

# Now importing the env file so the script can access the KuCoin API keys:
import env

# Defining the api keys with their own variables:
api_key = env.kc_api_key
api_s = env.kc_secret_api
api_pp = env.kc_passphrase
api_uid = env.kc_uid

# creating the api keys for use in the calls:
api_key = env.kc_api_key
api_secret = env.kc_secret_api
api_passphrase = env.kc_passphrase
url = 'https://api.kucoin.com/api/v1/accounts'
now = int(time.time() * 1000)
str_to_sign = str(now) + 'GET' + '/api/v1/accounts'
signature = base64.b64encode(
    hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
headers = {
    "KC-API-SIGN": signature,
    "KC-API-TIMESTAMP": str(now),
    "KC-API-KEY": api_key,
    "KC-API-PASSPHRASE": passphrase,
    "KC-API-KEY-VERSION": str(2)
}

# Getting the base response with the top level account values:
response = requests.request('get', url, headers=headers)


# Creating the account dataframe using the response request I just created:
df = pd.DataFrame.from_dict(response.json()['data'])

# Column cleanup:
df.drop(columns = 'id', inplace = True)

# Getting prices for coins:

coin_list = ['BTC', 'ETH', 'ADA', 'ALGO', 'ATOM', 'XLM', 'DOT']

# This for loop will create a list of prices by calling each crypto within my 'coin_list' list. 

price_list = []
for coin in coin_list:
    prices = float(requests.get(f'https://api.kucoin.com/api/v1/market/orderbook/level1?symbol={coin}-USDT').json()['data']['price'])
#     print(prices), print(type(prices))
    price_list.append(prices)

# Now creating a dictionary of the coin prices:
coin_dict = {"coin":coin_list, "price":price_list}

# Dataframe from the dictionary:
df_prices = pd.DataFrame(coin_dict)

# creating a copy dataframe of the acct info (this step can be dropped in future)
account = df[df['type'] == 'trade'].copy()
account = account.reset_index(drop = True)
account.rename(columns = {'type': "act_name"}, inplace = True)


# Now should be able to join the two dataframes. I have to join the two dataframes first before I can multiply columns to create the "$ value" column which is the end goal here.

# Merge, inner join:

holdings = account.merge(df_prices, left_on = 'currency', right_on = 'coin', how = 'left')

# Now changing the value types of the columns with numbers in them from objects to float64:
holdings['balance'] = holdings.balance.astype(float)
holdings['available'] = holdings.available.astype(float)
holdings['holds'] = holdings.holds.astype(float)

# Now working through adding calculated columns that I'll later select from if it doesn't return 'nan':

holdings['value_tmp'] = round(holdings.price * holdings.balance, 2)

holdings["dollar_value"] = np.where(holdings['value_tmp'].notnull(), holdings['value_tmp'], holdings['balance'])
holdings.dollar_value = holdings.dollar_value.round(2)

# Dropping extra columns:
holdings.drop(columns = ['coin', 'value_tmp'], inplace = True)

# printing out result, and saving to csv and Excel.
print(holdings)

# holdings.to_csv("account_holdings_index.csv")
# holdings.to_csv("account_holdings_no_index.csv", index = False)


# holdings.to_excel("account_holdings_index.xlsx")
# holdings.to_excel("account_holdings_no_index.xlsx", index = False)



  currency act_name      balance    available       holds       price  \
0     USDC    trade  1359.222089  1359.222089    0.000000         NaN   
1     USDT    trade   588.001716    53.244562  534.757154         NaN   
2     ATOM    trade    45.391100     0.237200   45.153900     11.7768   
3      BTC    trade     0.015401     0.015401    0.000000  34556.4000   
4     ALGO    trade     0.000000     0.000000    0.000000      0.8604   
5      ETH    trade     0.000000     0.000000    0.000000   2216.5400   
6    MATIC    trade     0.000000     0.000000    0.000000         NaN   

   dollar_value  
0       1359.22  
1        588.00  
2        534.56  
3        532.22  
4          0.00  
5          0.00  
6          0.00  


In [15]:
holdings

,currency,act_name,balance,available,holds,price,dollar_value
0,USDC,trade,1359.222089,1359.222089,0.000000,NaN,1359.22
1,USDT,trade,588.001716,53.244562,534.757154,NaN,588.00
2,ATOM,trade,45.391100,0.237200,45.153900,11.7768,534.56
3,BTC,trade,0.015401,0.015401,0.000000,34556.4000,532.22
4,ALGO,trade,0.000000,0.000000,0.000000,0.8604,0.00
5,ETH,trade,0.000000,0.000000,0.000000,2216.5400,0.00
6,MATIC,trade,0.000000,0.000000,0.000000,NaN,0.00


In [16]:
df.insert(0, 'TimeStamp', pd.to_datetime('now').replace(microsecond=0))
print (df)

             TimeStamp currency   type        balance      available  \
0  2021-07-03 13:20:36     USDC  trade  1359.22208948  1359.22208948   
1  2021-07-03 13:20:36     USDT  trade   588.00171574    53.24456156   
2  2021-07-03 13:20:36     ATOM  trade        45.3911         0.2372   
3  2021-07-03 13:20:36      BTC  trade     0.01540148     0.01540148   
4  2021-07-03 13:20:36     ALGO   main              0              0   
5  2021-07-03 13:20:36     ALGO  trade              0              0   
6  2021-07-03 13:20:36     ATOM   main              0              0   
7  2021-07-03 13:20:36      BTC   main              0              0   
8  2021-07-03 13:20:36      ETH   main              0              0   
9  2021-07-03 13:20:36      ETH  trade              0              0   
10 2021-07-03 13:20:36    MATIC   main              0              0   
11 2021-07-03 13:20:36    MATIC  trade              0              0   
12 2021-07-03 13:20:36     USDC   main              0           

In [17]:
df

,TimeStamp,currency,type,balance,available,holds
0,2021-07-03 13:20:36,USDC,trade,1359.22208948,1359.22208948,0
1,2021-07-03 13:20:36,USDT,trade,588.00171574,53.24456156,534.75715418
2,2021-07-03 13:20:36,ATOM,trade,45.3911,0.2372,45.1539
3,2021-07-03 13:20:36,BTC,trade,0.01540148,0.01540148,0
4,2021-07-03 13:20:36,ALGO,main,0,0,0
5,2021-07-03 13:20:36,ALGO,trade,0,0,0
6,2021-07-03 13:20:36,ATOM,main,0,0,0
7,2021-07-03 13:20:36,BTC,main,0,0,0
8,2021-07-03 13:20:36,ETH,main,0,0,0
9,2021-07-03 13:20:36,ETH,trade,0,0,0


In [18]:
datetime.datetime.strftime()

TypeError: descriptor 'strftime' of 'datetime.date' object needs an argument